<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.975 · Deep Learning · PEC4</p>
<p style="margin: 0; text-align:right;">2022-2 · Master universitario en Ciencia de datos (Data science)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informatica, Multimedia y Telecomunicaciones</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>


# PEC 4: Modelos generativos

En esta práctica implementaremos uno de los tipos de modelos generativos más utilizados actualmente, las redes generativas adversarias, ie. **GANs**.

<u>Consideraciones generales</u>:

- Esta PEC debe realizarse de manera **estrictamente individual**. Cualquier indicio de copia será penalizado con un suspenso (D) para todas las partes implicadas y la posible evaluación negativa de la asignatura de forma íntegra.
- Es necesario que el estudiante indique **todas las fuentes** que ha utilizado para la realización de la PEC. Si no es así, se considerará que el estudiante ha cometido plagio, siendo penalizado con un suspenso (D) y la posible evaluación negativa de la asignatura de forma íntegra.

<u>Formato de entrega</u>:

- Algunos ejercicios pueden suponer varios minutos de ejecución, por lo que la entrega debe realizarse en **formato notebook** y en **formato html**, donde se vea el código, los resultados y comentarios de cada ejercicio. Se puede exportar el notebook a HTML desde el menú File $\to$ Download as $\to$ HTML.
- Existe un tipo de celda especial para albergar texto. Este tipo de celda le será muy útil para responder a las diferentes preguntas teóricas planteadas a lo largo de la actividad. Puede cambiar el tipo de celda a este tipo, en el menú: Cell $\to$ Cell Type $\to$ Markdown.

# 0. Introducción

El objetivo de esta PEC es comprender la implementación de una solución generativa, utilizando DCGANs para la generación de imágenes, mediante el conjunto de datos de referencia en deep learning más sencillo existente: MNIST.


In [1]:
import numpy as np
import time
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Reshape
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import LeakyReLU, Dropout
from keras.layers import BatchNormalization
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot as plt

2023-05-24 15:17:59.165823: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-24 15:17:59.329596: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-24 15:18:00.174514: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-11.7/lib64
2023-05-24 15:18:00.174690: W tensorflow/compiler/xla/strea

# 1. Obtención de los datos

El código para cargar los datos es el siguiente:

In [2]:
latent_dim = 100

img_rows, img_cols = 28, 28
img_channels = 1
(x_train, _), (_, _) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, img_channels)
x_train = x_train.astype('float32')
x_train /= 255

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<p><strong>Ejercicio [1,5 pts.]:</strong> 
Añade un comentario explicativo, a cada una de las líneas de código de abajo, indicando cuál es su funcionalidad.</div>

**Respuesta**:

* `latent_dim = 100`: se define un espacio latente formado por un vector de 100 dimensiones que servirá al generador para generar las imágenes falsas.
* `img_rows, img_cols = 28, 28`: dimensiones de las imágenes   
* `img_channels = 1`: número de canales de la imagen (monocromo)
* `(x_train, _), (_, _) = mnist.load_data()`: función que permite cargar el conjunto de datos
* `x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, img_channels)`: Se añade la dimensión de los canales, que siendo de 1 es necesaria para la utilización de las librerías tensoriales.
* `x_train = x_train.astype('float32')`: los datos originales son enteros de 1 byte, se convierten en reales de 32 bits para poder realizar el entrenamiento/optimización
* `x_train /= 255`: Normalización de los datos en el intervalo [0,1]



## 2. Implementación del Generador

A continuación se muestra una propuesta de generador:

In [3]:
def generator_model(): 
    dropout = 0.4
    depth = 256 # 64+64+64+64
    dim = 7
    
    model = Sequential()
    # In: 100
    # Out: dim x dim x depth
    model.add(Dense(dim*dim*depth, input_dim=latent_dim))
    model.add(BatchNormalization(momentum=0.9))
    model.add(Activation('relu'))
    model.add(Reshape((dim, dim, depth)))
    model.add(Dropout(dropout))

    # In: dim x dim x depth
    # Out: 2*dim x 2*dim x depth/2
    model.add(UpSampling2D())
    model.add(Conv2DTranspose(int(depth/2), 5, padding='same'))
    model.add(BatchNormalization(momentum=0.9))
    model.add(Activation('relu'))

    model.add(UpSampling2D())
    model.add(Conv2DTranspose(int(depth/4), 5, padding='same'))
    model.add(BatchNormalization(momentum=0.9))
    model.add(Activation('relu'))

    model.add(Conv2DTranspose(int(depth/8), 5, padding='same'))
    model.add(BatchNormalization(momentum=0.9))
    model.add(Activation('relu'))

    # Out: 28 x 28 x 1 grayscale image [0.0,1.0] per pix
    model.add(Conv2DTranspose(1, 5, padding='same'))
    model.add(Activation('sigmoid'))
    
    return model

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<p><strong>Ejercicio [1,75 pts.]:</strong> 
Contesta a las preguntas siguientes:
</div>

**1. ¿Cuál es la finalidad del generador?:**

El generador sintetiza nuevas imágenes a partir de un ruido de 100 dimensiones (distribución uniforme entre -1,0 y 1,0) utilizando la inversa de la convolución, llamada convolución transpuesta.

**2. Investigar por qué se utiliza `Upsampling` en las dos primeras capas en lugar de la `Conv2DTranspose` propuesta en DCGAN. Dar una justificación:**

En vez de la convolución de pasos fraccionados, tal y como se sugiere en DCGAN, se utiliza el muestreo superior entre las tres primeras capas, sintetizando imágenes de escritura a mano más realistas.

**3. ¿Por qué se utiliza la normalización entre capas?**

Entre capas, la normalización por lotes estabiliza el aprendizaje.

**4. ¿Qué funciones de activación se utilizan? ¿Cuál es la razón de la sigmoide en la última capa?**

La función de activación después de cada capa es una ReLU. La salida del sigmoide en la última capa produce la imagen falsa. La razón de la sigmoide en la última capa se asegura que la salida esté en el intervalo [0,1]



## 3. Implementación del Discriminador

A continuación se muestra el discriminador propuesto

In [4]:
# (W−F+2P)/S+1
def discriminator_model():
    depth = 64
    dropout = 0.4
    input_shape = (img_rows, img_cols, img_channels)
    
    model = Sequential()
    # In: 28 x 28 x 1, depth = 1
    # Out: 14 x 14 x 1, depth=64
    model.add(Conv2D(depth, 5, strides=2, input_shape=input_shape, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(dropout))

    model.add(Conv2D(depth*2, 5, strides=2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(dropout))

    model.add(Conv2D(depth*4, 5, strides=2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(dropout))

    model.add(Conv2D(depth*8, 5, strides=1, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(dropout))

    # Out: 1-dim probability
    model.add(Flatten())
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    return model

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<p><strong>Exercici [1,75 pts.]:</strong> 
Contesta a las preguntas siguientes:
</div>


**1. ¿Cuál es la finalidad del discriminador?:**

Un discriminador que indica lo real que es una imagen, es básicamente una red neuronal convolucional profunda.

La función de activación utilizada en cada capa de CNN es una ReLU con fugas.
 

**2. ¿Cuáles son las dimensiones de los tensores y características de las variables de entrada y salida del discriminador? :**

Para el conjunto de datos MNIST, la entrada es una imagen (28 píxeles x 28 píxeles x 1 canal). La salida sigmoide es un valor escalar de la probabilidad de la realidad de la imagen (0,0 es ciertamente falso, 1,0 es real, cualquier cosa que hay en medio es un área gris).

**3. ¿Cuál es la diferencia con una CNN habitual?**

La diferencia con una CNN típica es la ausencia de max-pooling entre capas. En su lugar, se utiliza una convolución con stride para reducir la dimensionalidad.

**4. ¿Qué funciones de activación se utilizan?**

La función de activación utilizada en cada capa de CNN es una ReLU con fugas.

**5. ¿Cuál es la finalidad del dropout que encontramos en las capas?**

Dropout 0,4 y 0,7 entre capas evita el ajuste excesivo y la memorización.

# 4. Modelo GAN

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<p><strong>Ejercicio [1 pts.]:</strong> 
Contesta a les siguientes preguntas:
</div>


**1. ¿A qué llamamos modelo GAN y por qué recibe ese nombre?:**

Llamamos GAN al conjunto de Generador y Discriminador. La denominación "Generative Adversarial Network" se refiere a la interacción competitiva entre el generador y el discriminador en el proceso de aprendizaje de la red. El generador es responsable de crear nuevas muestras de datos, como imágenes o textos, a partir de un conjunto de datos de entrada. Por su parte, el discriminador tiene como tarea distinguir entre las muestras generadas por el generador y las muestras reales del conjunto de datos original.

## 4.1 Modelo Discriminador

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<p><strong>Ejercicio [1 pts.]:</strong> 
Contesta a las siguientes preguntas:
</div>


**1. ¿Qué función de pérdida utiliza el discriminador? ¿Por qué? :**

Dado que la salida del discriminador es sigmoide, utilizamos `binary cross entropy` para la pérdida.

**2. Busca en la bibliografía la razón por la que se propone utilizar `RMSProp` como optimizador en vez de otros.**
RMSProp como optimizador genera imágenes falsas más realistas en comparación con Adam para este caso.

**3. ¿Cuál es la razón de utilizar decay?**

El weight decay y el valor del clip estabilizan el aprendizaje durante la última parte del entrenamiento

In [5]:
discriminator = discriminator_model()
discriminator.compile(loss='binary_crossentropy', 
                      optimizer=RMSprop(lr=0.0002, decay=6e-8), 
                      metrics=['accuracy'])

2023-05-24 15:18:41.395272: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-24 15:18:41.439752: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-24 15:18:41.439913: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-24 15:18:41.440774: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operati

In [6]:
generator = generator_model()

## 4.2 Modelo adversario

El modelo adversario es sólo el generador-discriminador apilados juntos. Los parámetros de entrenamiento son los mismos que en el modelo Discriminador, salvo por una tasa de aprendizaje reducida y la correspondiente disminución del peso.

In [7]:
def adversarial_model():
    model = Sequential()
    model.add(generator)
    discriminator.trainable = False
    model.add(discriminator)
    model.compile(loss='binary_crossentropy', 
                  optimizer=RMSprop(lr=0.0001, decay=3e-8), 
                  metrics=['accuracy'])
    discriminator.trainable = True
    return model

In [8]:
adversarial = adversarial_model()

## 4.3 Entrenamiento

In [9]:
def plot_images(saveToFile=False, fake=True, samples=16, noise=None, epoch=0):
    filename = 'mnist.png'
    if fake:
        if noise is None:
            noise = np.random.uniform(-1.0, 1.0, size=[samples, latent_dim])
        else:
            filename = "mnist_%d.png" % epoch
        images = generator.predict(noise)
    else:
        i = np.random.randint(0, x_train.shape[0], samples)
        images = x_train[i, :, :, :]

    plt.figure(figsize=(10,10))
    for i in range(images.shape[0]):
        plt.subplot(4, 4, i+1)
        image = images[i, :, :, :]
        image = np.reshape(image, [img_rows, img_cols])
        plt.imshow(image, cmap='gray')
        plt.axis('off')
    plt.tight_layout()
    if saveToFile:
        plt.savefig(filename)
        plt.close('all')
    else:
        plt.show()

Primero determinamos si el modelo de discriminador es correcto entrenándolo solo con imágenes reales y falsas. Después, los modelos Discriminador y Adversario entrenan uno tras otro.

In [10]:
def train(train_epochs=2000, batch_size=256, save_interval=0):
        noise_input = None
        if save_interval>0:
            noise_input = np.random.uniform(-1.0, 1.0, size=[16, latent_dim])
        for epoch in range(train_epochs):
            
            # ---------------------
            #  Train Discriminator
            # ---------------------
            
            # select a random half of images
            images_train = x_train[np.random.randint(0, x_train.shape[0], size=batch_size), :, :, :]
            
            # sample noise and generate a batch of new images
            noise = np.random.uniform(-1.0, 1.0, size=[batch_size, latent_dim])
            images_fake = generator.predict(noise)
            
            # train the discriminator (real classified as ones and generated as zeros)
            x = np.concatenate((images_train, images_fake))
            y = np.ones([2*batch_size, 1])
            y[batch_size:, :] = 0
            d_loss = discriminator.train_on_batch(x, y)

            # ---------------------
            #  Train Generator
            # ---------------------
            
            # train the generator (wants discriminator to mistake images as real)
            y = np.ones([batch_size, 1])
            a_loss = adversarial.train_on_batch(noise, y)
            
            log_msg = "%d: [D loss: %f, acc: %f]" % (epoch, d_loss[0], d_loss[1])
            log_msg = "%s  [A loss: %f, acc: %f]" % (log_msg, a_loss[0], a_loss[1])
            print(log_msg)
            if save_interval>0:
                if (epoch+1)%save_interval==0:
                    plot_images(saveToFile=True, samples=noise_input.shape[0],
                                noise=noise_input, epoch=(epoch+1))

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<p><strong>Ejercicio [2 pts.]:</strong> 
Contesta a las siguientes preguntas:
</div>


**1. ¿Cuál es la finalidad de `noise = np.random.uniform(-1.0, 1.0, size=[batch_size, latent_dim])`? ¿Por qué estas dimensiones?**

La finalidad de esta línea es la generación de ruido aleatorio que servirá como punto de partida para que el generador genere una imagen. La primera de las dimensiones es el tamaño del `batch`, la segunda las dimensiones del vector de variables latentes.

**2. ¿Cuál es la finalidad de `images_fake = generator.predict(noise)`?**

Generar las imágenes fake por un batch a partir del generador.

**3. ¿Cuál es la finalidad del código que sigue?**
```python
x = np.concatenate((images_train, images_fake))
y = np.ondas([2*batch_size, 1])
y[batch_size:, :] = 0
```

Su finalidad es preparar un conjunto formado por imágenes reales e imágenes fake. En las fake se las etiqueta como 0, mientras que en las reales como 1. Esta información se utilizará para entrenar al modelo discriminador.

**4. ¿Qué realiza el comando `d_loss = discriminator.train_on_batch(x, y)`? ¿Qué devuelve?**

Este pedido ejecuta el entrenamiento del discriminador. Devuelve la función de pérdida asociada al error de entrenamiento.

**5. ¿Qué realiza el comando `a_loss = adversarial.train_on_batch(noise, y)`? ¿Qué devuelve?**

Este pedido realiza el entrenamiento del generador. Devuelve la función de pérdida asociada al error.



In [11]:
class ElapsedTimer(object):
    def __init__(self):
        self.start_time = time.time()
    def elapsed(self,sec):
        if sec < 60:
            return str(sec) + " sec"
        elif sec < (60 * 60):
            return str(sec / 60) + " min"
        else:
            return str(sec / (60 * 60)) + " hr"
    def elapsed_time(self):
        print("Elapsed: %s " % self.elapsed(time.time() - self.start_time))

In [ ]:
timer = ElapsedTimer()
train(train_epochs=1000, batch_size=256, save_interval=100) 
timer.elapsed_time()
plot_images(fake=True)
plot_images(fake=False, saveToFile=True)

2023-05-24 15:19:04.848117: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-05-24 15:19:05.041267: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8700


8/8 [==============================] - 3s 4ms/step


2023-05-24 15:19:06.961209: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-05-24 15:19:07.413763: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.14GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-05-24 15:19:07.413792: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.14GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-05-24 15:19:07.566421: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to a

0: [D loss: 0.693025, acc: 0.501953]  [A loss: 1.277778, acc: 0.000000]
8/8 [==============================] - 0s 4ms/step
1: [D loss: 0.590089, acc: 0.500000]  [A loss: 1.795848, acc: 0.000000]
8/8 [==============================] - 0s 4ms/step
2: [D loss: 0.344943, acc: 0.966797]  [A loss: 1.040250, acc: 0.160156]
8/8 [==============================] - 0s 4ms/step
3: [D loss: 0.105615, acc: 1.000000]  [A loss: 12.553440, acc: 0.000000]
8/8 [==============================] - 0s 4ms/step
4: [D loss: 1.348715, acc: 0.507812]  [A loss: 0.314578, acc: 0.972656]
8/8 [==============================] - 0s 4ms/step
5: [D loss: 0.069144, acc: 0.998047]  [A loss: 0.507411, acc: 0.789062]
8/8 [==============================] - 0s 4ms/step
6: [D loss: 0.045524, acc: 0.992188]  [A loss: 0.234772, acc: 0.980469]
8/8 [==============================] - 0s 4ms/step
7: [D loss: 0.044444, acc: 0.998047]  [A loss: 0.191159, acc: 0.972656]
8/8 [==============================] - 0s 4ms/step
8: [D loss: 0.0

8/8 [==============================] - 0s 4ms/step
67: [D loss: 0.034028, acc: 0.998047]  [A loss: 6.141680, acc: 0.000000]
8/8 [==============================] - 0s 4ms/step
68: [D loss: 0.032775, acc: 0.996094]  [A loss: 4.763631, acc: 0.000000]
8/8 [==============================] - 0s 4ms/step
69: [D loss: 0.034047, acc: 0.996094]  [A loss: 5.340449, acc: 0.000000]
8/8 [==============================] - 0s 4ms/step
70: [D loss: 0.024694, acc: 0.996094]  [A loss: 4.826488, acc: 0.000000]
8/8 [==============================] - 0s 3ms/step
71: [D loss: 0.032638, acc: 0.994141]  [A loss: 5.696912, acc: 0.000000]
8/8 [==============================] - 0s 4ms/step
72: [D loss: 0.029583, acc: 0.992188]  [A loss: 4.985974, acc: 0.000000]
8/8 [==============================] - 0s 4ms/step
73: [D loss: 0.037067, acc: 0.998047]  [A loss: 6.816721, acc: 0.000000]
8/8 [==============================] - 0s 4ms/step
74: [D loss: 0.057419, acc: 0.982422]  [A loss: 2.370356, acc: 0.019531]
8/8 [===

132: [D loss: 0.286841, acc: 0.953125]  [A loss: 1.532036, acc: 0.023438]
8/8 [==============================] - 0s 4ms/step
133: [D loss: 0.265263, acc: 0.964844]  [A loss: 1.499036, acc: 0.007812]
8/8 [==============================] - 0s 4ms/step
134: [D loss: 0.266981, acc: 0.957031]  [A loss: 1.702957, acc: 0.011719]
8/8 [==============================] - 0s 4ms/step
135: [D loss: 0.284436, acc: 0.935547]  [A loss: 1.270456, acc: 0.042969]
8/8 [==============================] - 0s 4ms/step
136: [D loss: 0.288526, acc: 0.923828]  [A loss: 2.375799, acc: 0.000000]
8/8 [==============================] - 0s 4ms/step
137: [D loss: 0.347714, acc: 0.894531]  [A loss: 0.626800, acc: 0.656250]
8/8 [==============================] - 0s 4ms/step
138: [D loss: 0.537502, acc: 0.605469]  [A loss: 3.457209, acc: 0.000000]
8/8 [==============================] - 0s 4ms/step
139: [D loss: 0.801939, acc: 0.605469]  [A loss: 0.503103, acc: 0.804688]
8/8 [==============================] - 0s 4ms/step


8/8 [==============================] - 0s 4ms/step
198: [D loss: 0.547146, acc: 0.644531]  [A loss: 1.854545, acc: 0.000000]
8/8 [==============================] - 0s 4ms/step
199: [D loss: 0.533715, acc: 0.671875]  [A loss: 0.656968, acc: 0.605469]
8/8 [==============================] - 0s 4ms/step
200: [D loss: 0.525931, acc: 0.658203]  [A loss: 1.730413, acc: 0.000000]
8/8 [==============================] - 0s 4ms/step
201: [D loss: 0.530167, acc: 0.705078]  [A loss: 0.728337, acc: 0.542969]
8/8 [==============================] - 0s 4ms/step
202: [D loss: 0.524917, acc: 0.666016]  [A loss: 1.702221, acc: 0.011719]
8/8 [==============================] - 0s 4ms/step
203: [D loss: 0.520530, acc: 0.714844]  [A loss: 0.738521, acc: 0.535156]
8/8 [==============================] - 0s 4ms/step
204: [D loss: 0.529219, acc: 0.667969]  [A loss: 1.707672, acc: 0.007812]
8/8 [==============================] - 0s 4ms/step
205: [D loss: 0.531775, acc: 0.726562]  [A loss: 0.746122, acc: 0.480469]


<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<p><strong>Ejercicio [1 pts.]:</strong> 
Contesta a las siguientes preguntas:
</div>

**1. Explica qué hacen las siguientes líneas de código: **

```python
timer = ElapsedTimer()
train(train_epochs=1000, batch_size=256, save_interval=100)
timer.elapsed_time()
plot_images(fake=True)
plot_images(fake=False, saveToFile=True)
```

Ponen en marcha un timer para contabilizar el tiempo total de entrenamiento, llevan a cabo el entrenamiento y finalmente guardan unas muestras de imágenes generadas y reales para ver si el modelo ha hecho su función.

**2. Escribe el código necesario para mostrar las imágenes generadas en la última iteración y muestra los resultados:**

# Referencias consultadas:

* Añadir